# Restaurants

In [1]:
# Import libraries
import pandas as pd
import numpy as np

In [2]:
# Import the data
df_business = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_business.csv')
df_attributes = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_business_attributes.csv')
df_hours = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_business_hours.csv')
df_checkin = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_checkin.csv')
df_review = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_review.csv')
df_tip = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_tip.csv')
df_user = pd.read_csv(r'C:\Users\jinho\Downloads\Mining\Projects\Team1\data\yelp_user.csv')

In [3]:
df_hours.columns

Index(['business_id', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday',
       'saturday', 'sunday'],
      dtype='object')

In [4]:
# Replace all 'Na' with NULL
df_business = df_business.replace('Na', np.nan, regex = True)
df_attributes = df_attributes.replace('Na', np.nan, regex = True)
df_hours = df_hours.replace('Na', np.nan, regex = True)
df_checkin = df_checkin.replace('Na', np.nan, regex = True)
df_review = df_review.replace('Na', np.nan, regex = True)
df_tip = df_tip.replace('Na', np.nan, regex = True)
df_user = df_user.replace('Na', np.nan, regex = True)

In [6]:
df_rating = df_business[['business_id', 'categories', 'stars', 'review_count']]
df_restaurants = df_rating[df_rating['categories'].str.contains('Restaurants|Food') == True]

## Effect of Business Hours

### Pre-Processing

In [133]:
star_hours = pd.merge(df_restaurants, df_hours, on = 'business_id', how = 'inner')

# Replace all 'None' with NULL & Remove them
star_hours = star_hours.replace('None', np.nan, regex = True)
star_hours = star_hours.dropna()

star_hours = star_hours.reset_index()
star_hours = star_hours.drop(columns = ['index'])

In [134]:
star_hours[['mon_op','mon_close']] = star_hours['monday'].str.split('-', expand = True)
star_hours[['tue_op','tue_close']] = star_hours['tuesday'].str.split('-', expand = True)
star_hours[['wed_op','wed_close']] = star_hours['wednesday'].str.split('-', expand = True)
star_hours[['th_op','th_close']] = star_hours['thursday'].str.split('-', expand = True)
star_hours[['fr_op','fr_close']] = star_hours['friday'].str.split('-', expand = True)
star_hours[['sat_op','sat_close']] = star_hours['saturday'].str.split('-', expand = True)
star_hours[['sun_op','sun_close']] = star_hours['sunday'].str.split('-', expand = True)
star_hours = star_hours.drop(columns = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday'])

In [135]:
# Striping minutes for the sake of simplicity
list_of_cols = list(star_hours.columns)[4:]

for i in range(len(star_hours)):
    for col in list_of_cols:
        head, sep, tail = star_hours.at[i,col].partition(':')
        star_hours.at[i,col] = head

In [137]:
star_hours['mon_time'] = star_hours['mon_close'].astype(int) - star_hours['mon_op'].astype(int)
star_hours.loc[star_hours['mon_time'] < 0, 'mon_time'] = 24 + star_hours['mon_close'].astype(int) - star_hours['mon_op'].astype(int)
star_hours['tue_time'] = star_hours['tue_close'].astype(int) - star_hours['tue_op'].astype(int)
star_hours.loc[star_hours['tue_time'] < 0, 'tue_time'] = 24 + star_hours['tue_close'].astype(int) - star_hours['tue_op'].astype(int)
star_hours['wed_time'] = star_hours['wed_close'].astype(int) - star_hours['wed_op'].astype(int)
star_hours.loc[star_hours['wed_time'] < 0, 'wed_time'] = 24 + star_hours['wed_close'].astype(int) - star_hours['wed_op'].astype(int)
star_hours['th_time'] = star_hours['th_close'].astype(int) - star_hours['th_op'].astype(int)
star_hours.loc[star_hours['th_time'] < 0, 'th_time'] = 24 + star_hours['th_close'].astype(int) - star_hours['th_op'].astype(int)
star_hours['fr_time'] = star_hours['fr_close'].astype(int) - star_hours['fr_op'].astype(int)
star_hours.loc[star_hours['fr_time'] < 0, 'fr_time'] = 24 + star_hours['fr_close'].astype(int) - star_hours['fr_op'].astype(int)
star_hours['sat_time'] = star_hours['sat_close'].astype(int) - star_hours['sat_op'].astype(int)
star_hours.loc[star_hours['sat_time'] < 0, 'sat_time'] = 24 + star_hours['sat_close'].astype(int) - star_hours['sat_op'].astype(int)
star_hours['sun_time'] = star_hours['sun_close'].astype(int) - star_hours['sun_op'].astype(int)
star_hours.loc[star_hours['sun_time'] < 0, 'sun_time'] = 24 + star_hours['sun_close'].astype(int) - star_hours['sun_op'].astype(int)

In [138]:
star_hours['weekly_time'] = star_hours['mon_time'] + star_hours['tue_time'] + star_hours['wed_time'] + star_hours['th_time'] + star_hours['fr_time'] + star_hours['sat_time'] + star_hours['sun_time']

In [140]:
star_hours.head()

,business_id,categories,stars,review_count,mon_op,mon_close,tue_op,tue_close,wed_op,wed_close,...,sun_op,sun_close,mon_time,tue_time,wed_time,th_time,fr_time,sat_time,sun_time,weekly_time
0,PfOCPjBrlQAnz__NXj9h_w,American (New);Nightlife;Bars;Sandwiches;Ameri...,3.5,116,11,1,11,1,11,1,...,11,0,14,14,14,14,14,15,13,98
1,l09JfMeQ6ynYs5MCJtrcmQ,Italian;French;Restaurants,3.0,12,9,22,9,22,9,22,...,9,22,13,13,13,13,13,13,13,91
2,Gu-xs3NIQTj3Mj2xYoN2aw,French;Food;Bakeries;Restaurants,3.5,34,11,22,11,22,11,22,...,11,22,11,11,11,11,12,12,11,79
3,lHYiCS-y8AFjUitv6MGpxg,Food;Coffee & Tea,4.0,21,5,22,5,22,5,22,...,7,22,17,17,17,17,18,17,15,118
4,1K4qrnfyzKzGgJPBEcJaNQ,Tiki Bars;Nightlife;Mexican;Restaurants;Bars,3.5,39,11,2,11,2,11,2,...,10,2,15,15,15,15,15,16,16,107


### Linear Regression

In [158]:
# Effect of time each day on stars
X = star_hours[['mon_time', 'tue_time', 'wed_time', 'th_time', 'fr_time', 'sat_time', 'sun_time']]
y = star_hours['stars']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 119)

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(X_train, y_train)

print('Coefficients:', model.coef_)

y_test_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
lm_mse = mean_squared_error(y_test, y_test_pred)
print('MSE:', lm_mse)

Coefficients: [-0.01512629  0.00716233 -0.00507305  0.02904103 -0.00350665  0.00244671
 -0.03828444]
MSE: 0.5643320316519558


In [160]:
# Effect of weekly_time on stars
X = star_hours[['weekly_time']]
y = star_hours['stars']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 119)

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(X_train, y_train)

print('Coefficients:', model.coef_)

y_test_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
lm_mse = mean_squared_error(y_test, y_test_pred)
print('MSE:', lm_mse)

Coefficients: [-0.00311994]
MSE: 0.568942467354237


In [161]:
# Effect of time each day on review_count
X = star_hours[['mon_time', 'tue_time', 'wed_time', 'th_time', 'fr_time', 'sat_time', 'sun_time']]
y = star_hours['review_count']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 119)

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(X_train, y_train)

print('Coefficients:', model.coef_)

y_test_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
lm_mse = mean_squared_error(y_test, y_test_pred)
print('MSE:', lm_mse)

Coefficients: [-2.56376676 -0.86740674  0.82144819 -6.44266379  1.26823347  3.21644183
  3.76166889]
MSE: 34163.393716284765


In [162]:
# Effect of weekly_time on stars
X = star_hours[['weekly_time']]
y = star_hours['review_count']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 119)

from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(X_train, y_train)

print('Coefficients:', model.coef_)

y_test_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error
lm_mse = mean_squared_error(y_test, y_test_pred)
print('MSE:', lm_mse)

Coefficients: [-0.13258439]
MSE: 34325.83036228876


### Ridge

In [163]:
# Effect of time each day on review_count
X = star_hours[['mon_time', 'tue_time', 'wed_time', 'th_time', 'fr_time', 'sat_time', 'sun_time']]
y = star_hours['review_count']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 119)

from sklearn.linear_model import Ridge

for i in range (1,10):
    ridge = Ridge(alpha = i)
    model = ridge.fit(X_train,y_train)
    y_test_pred = model.predict(X_test)
    print('Alpha = ',i,': MSE =', mean_squared_error(y_test, y_test_pred))

Alpha =  1 : MSE = 34163.39394161127
Alpha =  2 : MSE = 34163.3941674455
Alpha =  3 : MSE = 34163.394393786766
Alpha =  4 : MSE = 34163.3946206344
Alpha =  5 : MSE = 34163.39484798772
Alpha =  6 : MSE = 34163.39507584606
Alpha =  7 : MSE = 34163.39530420876
Alpha =  8 : MSE = 34163.39553307513
Alpha =  9 : MSE = 34163.395762444496


In [164]:
# Effect of weekly_time on stars
X = star_hours[['weekly_time']]
y = star_hours['review_count']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 119)

from sklearn.linear_model import Ridge
for i in range (1,10):
    ridge = Ridge(alpha = i)
    model = ridge.fit(X_train,y_train)
    y_test_pred = model.predict(X_test)
    print('Alpha = ', i, ': MSE =', mean_squared_error(y_test, y_test_pred))

Alpha =  1 : MSE = 34325.83036318189
Alpha =  2 : MSE = 34325.83036407503
Alpha =  3 : MSE = 34325.830364968155
Alpha =  4 : MSE = 34325.830365861286
Alpha =  5 : MSE = 34325.83036675442
Alpha =  6 : MSE = 34325.83036764755
Alpha =  7 : MSE = 34325.83036854068
Alpha =  8 : MSE = 34325.83036943382
Alpha =  9 : MSE = 34325.83037032694


### LASSO

In [166]:
# Effect of time each day on review_count
X = star_hours[['mon_time', 'tue_time', 'wed_time', 'th_time', 'fr_time', 'sat_time', 'sun_time']]
y = star_hours['review_count']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 119)

from sklearn.linear_model import Lasso
for i in range (1,10):
    lasso = Lasso(alpha = i)
    model = lasso.fit(X_train,y_train)
    y_test_pred = model.predict(X_test)
    print('Alpha = ', i, ': MSE =',mean_squared_error(y_test, y_test_pred))

Alpha =  1 : MSE = 34169.35016729976
Alpha =  2 : MSE = 34172.706555621524
Alpha =  3 : MSE = 34179.39017486131
Alpha =  4 : MSE = 34189.39366797368
Alpha =  5 : MSE = 34202.697626012494
Alpha =  6 : MSE = 34219.3315615729
Alpha =  7 : MSE = 34239.29479276272
Alpha =  8 : MSE = 34258.59013100671
Alpha =  9 : MSE = 34276.865867627545


In [167]:
# Effect of weekly_time on stars
X = star_hours[['weekly_time']]
y = star_hours['review_count']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 119)

from sklearn.linear_model import Lasso
for i in range (1,10):
    lasso = Lasso(alpha = i)
    model = lasso.fit(X_train,y_train)
    y_test_pred = model.predict(X_test)
    print('Alpha = ', i, ': MSE =',mean_squared_error(y_test, y_test_pred))

Alpha =  1 : MSE = 34326.004859835164
Alpha =  2 : MSE = 34326.18203833973
Alpha =  3 : MSE = 34326.361897802475
Alpha =  4 : MSE = 34326.54443822339
Alpha =  5 : MSE = 34326.72965960249
Alpha =  6 : MSE = 34326.91756193975
Alpha =  7 : MSE = 34327.10814523519
Alpha =  8 : MSE = 34327.301409488806
Alpha =  9 : MSE = 34327.4973547006
